In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

## Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence. You should end up with something like this:

<code>name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64</code>

In [2]:
# Read in attendance.csv
df = pd.read_csv('untidy-data/attendance.csv')
df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
# Melt
df = df.rename(columns={'Unnamed: 0':'name'})
df_melt = df.melt(id_vars='name', var_name='date', value_name='status')
df_melt.head()

,name,date,status
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [4]:
# Discover attendance status
# df_melt.status.unique()

# Assign new values
df_melt = df_melt.replace(to_replace={'status':{'P':1, 'A':0, 'T':0.9, 'H':0.5}})
df_melt.head()

,name,date,status
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [5]:
# Calculate attendance
df_tidy = df_melt.groupby('name').status.mean()
df_tidy

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: status, dtype: float64

## Coffee Levels

Read the coffee_levels.csv file.
Transform the data so that each carafe is in it's own column.
Is this the best shape for the data?

In [6]:
# Read in coffee_levels
df = pd.read_csv('untidy-data/coffee_levels.csv')
df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [7]:
# Pivot table of carafe
df = df.pivot_table(index='hour', columns='coffee_carafe', values='coffee_amount').reset_index()
df.head()

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


In [8]:
df.shape

(10, 4)

## Cake Recipes

Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
Tidy the data as necessary.
Which recipe, on average, is the best? recipe b
Which oven temperature, on average, produces the best results? 275
Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [9]:
# Read in cake_recipes
df = pd.read_csv('untidy-data/cake_recipes.csv')
df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [10]:
# Tidy data - Melt
df_melt = df.melt(id_vars='recipe:position', var_name='temp', value_name='score')
df_melt.head()

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [11]:
# Tidy data - Split recipe:position
df_melt[['recipe', 'position']] = df_melt['recipe:position'].str.split(':', expand=True)
df_melt = df_melt.drop(columns='recipe:position')
df_melt.head()

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom


#### Which recipe, on average, is the best? 
recipe b 

In [12]:
df_tidy = df_melt.copy()
df_tidy.groupby('recipe').score.mean().sort_values(ascending=False).head(1)

recipe
b    76.736074
Name: score, dtype: float64

#### Which oven temperature, on average, produces the best results? 
275 

In [13]:
df_tidy.groupby('temp').score.mean().sort_values(ascending=False).head(1)

temp
275    74.886754
Name: score, dtype: float64

#### Which combination of recipe, rack position, and temperature gives the best result? 
recipe b, bottom rack, 300 degrees

In [14]:
df_tidy.sort_values(by='score', ascending=False).head(1)

,temp,score,recipe,position
26,300,99.248541,b,bottom


## Bonus - Tidying Gapminder1, Gapminder2

### Gapminder1

In [15]:
df = pd.read_csv('untidy-data/gapminder1.csv')
df.head()

,year,country,measure,measurement
0,1955,Afghanistan,pop,8891209.0
1,1960,Afghanistan,pop,9829450.0
2,1965,Afghanistan,pop,10997885.0
3,1970,Afghanistan,pop,12430623.0
4,1975,Afghanistan,pop,14132019.0


In [16]:
df.measure.unique()

array(['pop', 'life_expect', 'fertility'], dtype=object)

In [17]:
df_pivoted = df.pivot_table(index=['country', 'year'], 
                            columns='measure', 
                            values='measurement').reset_index()
df_pivoted.head()

measure,country,year,fertility,life_expect,pop
0,Afghanistan,1955,7.7,30.332,8891209.0
1,Afghanistan,1960,7.7,31.997,9829450.0
2,Afghanistan,1965,7.7,34.020,10997885.0
3,Afghanistan,1970,7.7,36.088,12430623.0
4,Afghanistan,1975,7.7,38.438,14132019.0


### Gapminder2

In [18]:
df = pd.read_csv('untidy-data/gapminder2.csv')
df.head()

,country,life_expect_1955,life_expect_1960,life_expect_1965,life_expect_1970,life_expect_1975,life_expect_1980,life_expect_1985,life_expect_1990,life_expect_1995,...,pop_1960,pop_1965,pop_1970,pop_1975,pop_1980,pop_1985,pop_1990,pop_1995,pop_2000,pop_2005
0,Afghanistan,30.332,31.997,34.020,36.088,38.438,39.854,40.822,41.674,41.763,...,9829450,10997885,12430623,14132019,15112149,13796928,14669339,20881480,23898198,29928987
1,Argentina,64.399,65.142,65.634,67.065,68.481,69.942,70.774,71.868,73.275,...,20616009,22283100,23962313,26081880,28369799,30675059,33022202,35311049,37497728,39537943
2,Aruba,64.381,66.606,68.336,70.941,71.830,74.116,74.494,74.108,73.011,...,57203,59020,59039,59390,60266,64129,66653,67836,69539,71566
3,Australia,70.330,70.930,71.100,71.930,73.490,74.740,76.320,77.560,78.830,...,10361273,11439384,12660160,13771400,14615900,15788300,17022133,18116171,19164620,20090437
4,Austria,67.480,69.540,70.140,70.630,72.170,73.180,74.940,76.040,77.510,...,7047437,7270889,7467086,7578903,7549433,7559776,7722953,8047433,8113413,8184691


In [19]:
df_melt = df.melt(id_vars='country')
df_melt.head()

,country,variable,value
0,Afghanistan,life_expect_1955,30.332
1,Argentina,life_expect_1955,64.399
2,Aruba,life_expect_1955,64.381
3,Australia,life_expect_1955,70.330
4,Austria,life_expect_1955,67.480


In [20]:
df_melt['year'] = df_melt.variable.str[-4:]
df_melt.head()

,country,variable,value,year
0,Afghanistan,life_expect_1955,30.332,1955
1,Argentina,life_expect_1955,64.399,1955
2,Aruba,life_expect_1955,64.381,1955
3,Australia,life_expect_1955,70.330,1955
4,Austria,life_expect_1955,67.480,1955


In [21]:
df_melt['measure'] = df_melt.variable.str[:-5]
df_melt.head()

,country,variable,value,year,measure
0,Afghanistan,life_expect_1955,30.332,1955,life_expect
1,Argentina,life_expect_1955,64.399,1955,life_expect
2,Aruba,life_expect_1955,64.381,1955,life_expect
3,Australia,life_expect_1955,70.330,1955,life_expect
4,Austria,life_expect_1955,67.480,1955,life_expect


In [22]:
df_melt.drop(columns='variable', inplace=True)
df_melt.head()

,country,value,year,measure
0,Afghanistan,30.332,1955,life_expect
1,Argentina,64.399,1955,life_expect
2,Aruba,64.381,1955,life_expect
3,Australia,70.330,1955,life_expect
4,Austria,67.480,1955,life_expect


In [23]:
df_tidy = df_melt.pivot_table(index=['country', 'year'], 
                              columns='measure',
                              values='value').reset_index()

df_tidy.head()

measure,country,year,life_expect,pop
0,Afghanistan,1955,30.332,8891209.0
1,Afghanistan,1960,31.997,9829450.0
2,Afghanistan,1965,34.020,10997885.0
3,Afghanistan,1970,36.088,12430623.0
4,Afghanistan,1975,38.438,14132019.0
